In [ ]:
import osimport numpy as npimport scipy.ioimport pandas as pdimport matplotlib.pyplot as pltfrom scipy.cluster.hierarchy import linkage, fclusterfrom scipy.integrate import simpsonfrom sklearn.metrics import jaccard_scorefrom nilearn import plotting# === PATHS ===data_dir = './data'pip_dir = './results/PiP_matrices'fig_dir = './figures/PiP_group_consensus'os.makedirs(fig_dir, exist_ok=True)# === Load coordinates ===coords_path = os.path.join(data_dir, 'MNI_66_coords.txt')coords = pd.read_csv(coords_path, sep="\t", header=None).values[:, :3]n_nodes = coords.shape[0]

In [ ]:
# === Load percolation participation matrices ===pip_paths = sorted([    os.path.join(pip_dir, f) for f in os.listdir(pip_dir)    if f.endswith('.mat')])# === Initialize vote counter ===consensus_votes = np.zeros(n_nodes)# === Process each subject's percolation matrix ===for path in pip_paths:    mat = scipy.io.loadmat(path)    if 'node_participation_at_percolation' not in mat:        continue    x = mat['node_participation_at_percolation'].T  # (nodes, steps)    # --- Crop to longest non-NaN block ---    x_non_nan = ~np.isnan(np.sum(x, axis=0))    D = np.diff(np.concatenate([[0], x_non_nan.astype(int), [0]]))    starts = np.where(D == 1)[0]    ends = np.where(D == -1)[0]    if len(starts) == 0 or len(ends) == 0:        continue    longest = np.argmax(ends - starts)    x_crop = x[:, starts[longest]:ends[longest]]    x_crop = np.nan_to_num(x_crop)    # --- Flatten and weight ---    flattening = -np.linspace(1 / x_crop.shape[1], 1, x_crop.shape[1])    weights = np.linspace(1, 1 / x_crop.shape[1], x_crop.shape[1])    x_flat = x_crop + flattening[np.newaxis, :]    x_weighted = x_flat * weights[np.newaxis, :]    x_weighted = np.clip(x_weighted, 0, None)    # --- Cluster nodes (Ward + AUC) ---    Z = linkage(x_weighted, method='ward')    labels = fcluster(Z, 2, criterion='maxclust')    auc_vals = np.array([simpson(x_weighted[i, :]) for i in range(x_weighted.shape[0])])    crit_label = np.argmax([np.mean(auc_vals[labels == l]) for l in np.unique(labels)])    crit_nodes = np.where(labels == crit_label + 1)[0]    consensus_votes[crit_nodes] += 1# === Normalize percolation-based consensus ===n_subs = len(pip_paths)consensus_score = consensus_votes / n_subs

In [ ]:
# === Load hub metrics ===hub_data = scipy.io.loadmat('./results/graph_hub_metrics/hub_metrics_all_subjects.mat')deg_all = hub_data['deg_all']bc_all = hub_data['bc_all']pr_all = hub_data['pr_all']# === Consensus clustering function ===def run_consensus_clustering(metric_matrix, k):    n_subs, n_nodes = metric_matrix.shape    consensus_votes = np.zeros(n_nodes)    for s in range(n_subs):        x = (metric_matrix[s, :] - np.mean(metric_matrix[s, :])) / np.std(metric_matrix[s, :])        Z = linkage(x[:, None], method='ward')        labels = fcluster(Z, k, criterion='maxclust')        means = [np.mean(x[labels == l]) for l in np.unique(labels)]        crit_label = np.argmax(means) + 1        crit_nodes = np.where(labels == crit_label)[0]        consensus_votes[crit_nodes] += 1    return consensus_votes / n_subsdef binary_top_15(score):    threshold = np.percentile(score, 85)    return (score >= threshold).astype(int)

In [ ]:
jaccard_records = []for k in [2, 3]:    print(f"\n--- Cluster size: {k} ---")    deg_score = run_consensus_clustering(deg_all, k)    bc_score = run_consensus_clustering(bc_all, k)    pr_score = run_consensus_clustering(pr_all, k)    # === Binarize top 15% ===    crit_bin = binary_top_15(consensus_score)    deg_bin = binary_top_15(deg_score)    bc_bin = binary_top_15(bc_score)    pr_bin = binary_top_15(pr_score)    # === Jaccard comparison ===    jaccard_deg = jaccard_score(crit_bin, deg_bin)    jaccard_bc = jaccard_score(crit_bin, bc_bin)    jaccard_pr = jaccard_score(crit_bin, pr_bin)    print(f"Jaccard w/ Degree     (k={k}): {jaccard_deg:.3f}")    print(f"Jaccard w/ Betweenness(k={k}): {jaccard_bc:.3f}")    print(f"Jaccard w/ PageRank   (k={k}): {jaccard_pr:.3f}")    jaccard_records.append({'Cluster_k': k, 'Metric': 'Degree', 'Jaccard': jaccard_deg})    jaccard_records.append({'Cluster_k': k, 'Metric': 'Betweenness', 'Jaccard': jaccard_bc})    jaccard_records.append({'Cluster_k': k, 'Metric': 'PageRank', 'Jaccard': jaccard_pr})    # === Plotting ===    for name, score in zip(['Percolation', 'Degree', 'Betweenness', 'PageRank'],                           [consensus_score, deg_score, bc_score, pr_score]):        top_nodes = binary_top_15(score)        highlight_vals = score * top_nodes        display = plotting.plot_markers(            node_values=highlight_vals,            node_coords=coords,            node_cmap='Reds',            display_mode='lzr'        )        out_path = os.path.join(fig_dir, f"{name.lower()}_consensus_k{k}.png")        display.savefig(out_path, dpi=300)        display.close()jaccard_df = pd.DataFrame(jaccard_records)jaccard_df